<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Ответьте-на-вопросы" data-toc-modified-id="Ответьте-на-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ответьте на вопросы</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Исследование надёжности заёмщиков - анализ банковских данных

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


 Получим общую информацию о таблице:

In [4]:
df.info()# общая информация о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов с разными типами данных.

Согласно документации к данным: 

* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Вывод**

В каждой строке таблицы - данные о клиенте: сколько ему лет, сколько детей и тд.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, которые необходимо устранить.


## Предобработка данных

In [5]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В столбцах `days_employed` и `total_income` отображен рабочий стаж в днях и доход в месяц соотвественно, обнаружены пропуски, которые необходимо обработать. Для начала заменим отрицательные значения в столбце `days_employed` на положительные, имеются аномально высокие значения. Появление аномально высоких значений рабочего стажа может быть связана с тем, что данные собранны с нескольких баз и счет идет не в днях, а в часах. При просмотре данных можно заметить, что аномально высокие значения почти все отрицательные. Установим верхнюю границу для пересчета аномальных значений в дни.

In [6]:
df['days_employed'] = df['days_employed'].apply(abs)
df.loc[df['days_employed']>30000, 'days_employed'] = df.loc[df['days_employed']>30000, 'days_employed'] / 24


Для замены пропусков в столбце `days_employed` воспользуемся медианой с разбивкой по типу занятости.

In [7]:
df_grouped = df.groupby('income_type')['days_employed'].median()

def median_group(row):
    if pd.isna(row['days_employed']):
        return df_grouped.loc[row['income_type']]
    return row['days_employed']

df['days_employed'] = df.apply(median_group, axis=1)

Так же заполняем пропуски в столбце `total_income` медианой с разбивкой по типу занятости и проверяем имеются ли пропуски.

In [8]:
df_grouped = df.groupby('income_type')['total_income'].median()

def median_group(row):
    if pd.isna(row['total_income']):
        return df_grouped.loc[row['income_type']]
    return row['total_income']

df['total_income'] = df.apply(median_group, axis=1)

In [9]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Приведем столбец `education` к нижнему регистру, для того чтобы разный регистр в столбце не мешал поиску явных дубликатов.

In [10]:
df['education'] = df['education'].str.lower() 

В столбце `children` есть отрицательные и аномально высокие значения. Значение 20 заменим на ноль, так как цифра 2 на клавиатуре очень близко находится с цифрой 0 и можно легко ощибиться.  - 1 тоже заменим на ноль, так это скорее всего означало прочерк, следовательно отсутствие детей.

In [11]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Заменим аномальные значения на 0. Проверим, прошла ли замена успешно.

In [12]:
df.loc[(df['children']==-1)|(df['children']==20), 'children'] = 0
df['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод** 
При обработке данных было несколько проблем:
- пропуски данных
- разный регистр в столбце
- аномальные значения в нескольких столбцах.

### Замена типа данных

Для того чтобы убрать ряд значений после запятой, поменяем вещественный тип данных столбцов  `days_employed` и `total_income` на целочисленный.

In [13]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод** Изменение типа данных прошло успешно.

### Обработка дубликатов

Посчитаем явные дубликаты в таблице

In [14]:
df.duplicated().sum()

71

Вызовем метод pandas для удаления дубликатов. Сразу удалим старые индексы и проверим есть ли еще явные дубликаты.

In [15]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

**Вывод** Явных дубликатов больше нет. Дубликаты в данных могли появится при неправильной выгрузке данных или данные были внесены неколько раз. 

### Лемматизация

Подключим библиотеку `pymystem3` для поиска лемм в столбце `purpose`. 

In [16]:
from pymystem3 import Mystem 
m = Mystem()

lemmas = m.lemmatize(' '.join(df['purpose'].unique()))

Посмотрим какие леммы встречаются чаще всего.

In [17]:
from collections import Counter
Counter(lemmas).most_common()

[(' ', 96),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('подержать', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('со', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1),
 ('\n', 1)]

Создадим список самых популярных вариантов.

In [18]:
common_purposes = ['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']

Создадим функцию для обработки столбца `purpose`, которая присвоит категорию каждой строке.

In [19]:
def purpose_category(value):
    lemmas = m.lemmatize(value)
    for item in common_purposes:
        if item in lemmas:
            return item
    return 'другое'

df['purpose_category'] = df['purpose'].apply(purpose_category)# добавляем новый столбец в df
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


**Вывод** С помощью лемматизации каждой строке присвоена категория для удобства анализа данных.

### Категоризация данных

Определим, по каким категорям можно разделить датафрейм:

1. По семейному статусу:

In [20]:
df[['family_status_id', 'family_status']].drop_duplicates().set_index('family_status_id')

,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


2. По уровню образования:

In [21]:
df[['education_id', 'education']].drop_duplicates().set_index('education_id')

,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


3. По уровню дохода:

In [22]:
df['total_income_group'] = pd.qcut(df['total_income'], [0, 0.2, 0.4, 0.6, 0.8, 1], 
                                   labels=['низкий доход', 'доход ниже среднего','средний доход', 'доход выше среднего', 'высокий доход'])
df['total_income_group'].value_counts()

низкий доход           4291
доход ниже среднего    4291
доход выше среднего    4291
высокий доход          4291
средний доход          4290
Name: total_income_group, dtype: int64

4. По цели получения кредита:

In [23]:
df['purpose_category'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_category, dtype: int64

## Ответьте на вопросы

Ответим на несколько вопросов:
    
*1. Есть ли зависимость между наличием детей и возвратом кредита в срок?*

In [24]:
children_pivot = df.pivot_table(index='children', columns='debt', values='education', aggfunc='count').fillna(0)
children_pivot.columns = ['no_debt', 'debt']
children_pivot['%'] = children_pivot['debt'] / (children_pivot['debt'] + children_pivot['no_debt'])
children_pivot.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})

,no_debt,debt,%
children,,,
0,13142,1072.000000,7.54%
1,4364,444.000000,9.23%
2,1858,194.000000,9.45%
3,303,27.000000,8.18%
4,37,4.000000,9.76%
5,9,0.000000,0.00%


In [25]:
children_pivot_wk =  df.loc[(df['children']>=1)&(df['debt']==1)]['children'].count()/df.loc[(df['children']>=1)]['children'].count()
#сложим и поделим все занчения с детьми
display(f'Процент клиентов с детьми, не вернувшие креди в срок {children_pivot_wk:.2%}')

'Процент клиентов с детьми, не вернувшие креди в срок 9.24%'

**Вывод** 
По данным в таблице видно, что процент задержек по выплате кредита у клиентов без детей - 7.54%. Самый большой процент задержек у клиентов с 4-мя детьми - 9.76%. Так же если сравнивать клиентов с детьми и без детей (9,24 и 7,54% соответственно), видно, что у клиентов без детей процент задержек меньше.

*2. Есть ли зависимость между семейным положением и возвратом кредита в срок?*

In [26]:
family_status_pivot = df.pivot_table(index=['family_status'], columns='debt', values='education', aggfunc='count').fillna(0)
family_status_pivot.columns = ['no_debt', 'debt']
family_status_pivot['%'] = family_status_pivot['debt'] / (family_status_pivot['debt'] + family_status_pivot['no_debt'])
family_status_pivot.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})

,no_debt,debt,%
family_status,,,
Не женат / не замужем,2536,274,9.75%
в разводе,1110,85,7.11%
вдовец / вдова,896,63,6.57%
гражданский брак,3763,388,9.35%
женат / замужем,11408,931,7.55%


**Вывод**  В категориях `Не женат/ не замужем` и `гражданский брак` процент невозврата кредита в срок выше остальных, а в категории `вдовец/вдова` процент задержек самый низкий.


*3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?*

In [27]:
total_income_group_pivot = df.pivot_table(index='total_income_group', columns='debt', values='education', aggfunc='count').fillna(0)
total_income_group_pivot.columns = ['no_debt', 'debt']
total_income_group_pivot['%'] = total_income_group_pivot['debt'] / (total_income_group_pivot['debt'] + total_income_group_pivot['no_debt'])
total_income_group_pivot.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})

,no_debt,debt,%
total_income_group,,,
низкий доход,3947,344,8.02%
доход ниже среднего,3930,361,8.41%
средний доход,3915,375,8.74%
доход выше среднего,3930,361,8.41%
высокий доход,3991,300,6.99%


**Вывод** Можно заметить, что процент задержек у клиентов с `высоким доходом` меньше, тогда как у клиентов со `средним доходом` процент задержек самый высокий.

*4. Как разные цели кредита влияют на его возврат в срок?*

In [28]:
purpose_category_pivot = df.pivot_table(index='purpose_category', columns='debt', values='education', aggfunc='count').fillna(0)
purpose_category_pivot.columns = ['no_debt', 'debt']
purpose_category_pivot['%'] = purpose_category_pivot['debt'] / (purpose_category_pivot['debt'] + purpose_category_pivot['no_debt'])
purpose_category_pivot.style.format({'%':'{:.2%}', 'no_debt':'{:.0f}'})

,no_debt,debt,%
purpose_category,,,
автомобиль,3903,403,9.36%
жилье,4152,308,6.91%
недвижимость,5877,474,7.46%
образование,3643,370,9.22%
свадьба,2138,186,8.00%


**Вывод** Самый низкий процент задержек у клиентов из категории `жилье`, самый высокий из категории `автомобиль`.

## Общий вывод

В ходе анализа данных проверили несколько гипотез:
    
    1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
    Судя по полученным цифрам, процент задержек у клиетнов имеющих детей выше(9,24%), чем у клиентов без детей(7,54%).
    
    2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
    Не женатые(9,75%) и живущие в гражданском браке люди(9,35%) имеют самый высокий процент задержек, тогда как в категории вдовец/вдова(6,57%) процент задержек самый низкий.
    
    3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    Клиенты со `средним доходом` имеют самый высокий процент задержек(8,67%), а `высоким  доходом` самый низкий процет(6,99%).
    
    4. Как разные цели кредита влияют на его возврат в срок?
    Самый низкий процент задержек у клиентов, которые брали кредит на жилье(6,91%). Самый высокий процент задержек у клиентов, которые брали кредит на автобомиль(9,36%).